In [1]:
import os
import numpy as np
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D,MaxPool2D
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import pickle

In [2]:
#Dataset: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
train_path = "chest_xray/train/"
test_path = "chest_xray/test/"
validation_path = "chest_xray/val/"

In [3]:
def process_data(img_dims, batch_size):
    # create three Data generation objects for train test and validation
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255) 

    # Code for train,val,test batches which will be fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=train_path, 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)

    test_gen = test_datagen.flow_from_directory(
    directory=test_path, 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)

    val_gen = val_datagen.flow_from_directory(
    directory=validation_path, 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)
    
    return train_gen,val_gen,test_gen

In [4]:
img_dims = 150
epochs = 10
batch_size = 32

# Getting the data
train_gen, val_gen , test_gen = process_data(img_dims, batch_size)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30,30))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
imgs, labels = next(train_gen)
plotImages(imgs)
print(labels)

In [ ]:
def model():
    model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(150,150,3)),
    BatchNormalization(),  #--------------batch normalization--------#
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation="relu", padding='same'),
    BatchNormalization(),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation="relu", padding='same'),  
    BatchNormalization(),
    MaxPool2D(pool_size=(3, 3)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(60, activation='relu'),
    Dropout(rate=0.2),    #---------dropout-----------------------#
    Dense(1, activation='sigmoid') ,

    ])

    return model

In [8]:
model = model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 150, 150, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 75, 75, 64)       256       
 hNormalization)                                                 
                                                        

In [9]:
model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics="accuracy")


In [10]:
es = EarlyStopping(patience=15, monitor='val_accuracy', restore_best_weights=True)

hist= model.fit(train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=epochs, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size,
           verbose=2,callbacks=[es])

Epoch 1/10


2023-03-20 01:42:20.663394: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


163/163 - 58s - loss: 0.2173 - accuracy: 0.9331 - val_loss: 11.4991 - val_accuracy: 0.6250 - 58s/epoch - 354ms/step
Epoch 2/10
163/163 - 61s - loss: 0.0948 - accuracy: 0.9655 - val_loss: 8.5816 - val_accuracy: 0.6283 - 61s/epoch - 374ms/step
Epoch 3/10
163/163 - 63s - loss: 0.0732 - accuracy: 0.9739 - val_loss: 2.4836 - val_accuracy: 0.6711 - 63s/epoch - 387ms/step
Epoch 4/10
163/163 - 64s - loss: 0.0647 - accuracy: 0.9774 - val_loss: 7.9780 - val_accuracy: 0.6283 - 64s/epoch - 392ms/step
Epoch 5/10
163/163 - 63s - loss: 0.0499 - accuracy: 0.9818 - val_loss: 13.4198 - val_accuracy: 0.6283 - 63s/epoch - 387ms/step
Epoch 6/10
163/163 - 63s - loss: 0.0478 - accuracy: 0.9816 - val_loss: 3.4738 - val_accuracy: 0.7039 - 63s/epoch - 388ms/step
Epoch 7/10
163/163 - 63s - loss: 0.0357 - accuracy: 0.9858 - val_loss: 7.0061 - val_accuracy: 0.6283 - 63s/epoch - 389ms/step
Epoch 8/10
163/163 - 63s - loss: 0.0458 - accuracy: 0.9826 - val_loss: 4.4880 - val_accuracy: 0.6480 - 63s/epoch - 387ms/step
E

In [11]:
pickle.dump(model,open('pneumonia_model.pkl','wb'))
model = pickle.load(open('pneumonia_model.pkl','rb'))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dropout
.........vars
......flatten
.........vars
......max_pooling2d
.........vars
......max_pooling2d_1
.........vars
......max_pooling2d_2
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...

In [20]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img_path = 'chest_xray/test/PNEUMONIA/person1_virus_6.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img = np.array(img)
img = np.expand_dims(img, axis=0)
img = img/255.0

# make a prediction
prediction = model.predict(img)[0]
print(prediction)
if prediction > 0.5:
    prediction = 'pneumonia'
else:
    prediction = 'normal'
print(prediction)

1/1 [==============================] - 0s 20ms/step
[1.]
pneumonia
